In [1]:
# when using google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [1]:
dataset_path = "/content/gdrive/MyDrive/Kelompok E Anamedsos/dataset douglas/query_tweet/"
path = "/content/gdrive/MyDrive/Kelompok E Anamedsos/CODE/1 - STANCE DETECTION/"

In [2]:
!pip install nltk
!pip install plotly
!pip install gensim
!pip install tqdm
!pip install xgboost

import numpy as np
import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import OrderedDict, Counter
from wordcloud import WordCloud
from tqdm import tqdm

from gensim import corpora, models, similarities
from gensim.models import CoherenceModel

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

sns.set_style("darkgrid")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data Collection
Data based on GitHub

In [4]:
import os

In [ ]:
list_filename = os.listdir(dataset_path)
list_data = []

for filename in list_filename:
  data = pd.read_csv(dataset_path+filename)
  if data.shape[0] > 0:
    list_data.append(data)

In [ ]:
df = pd.concat(list_data, ignore_index=True, sort=False)

In [ ]:
df.shape

(9627, 2)

In [ ]:
df.sample(5)

,id,tweet
6732,[1450150666299527169],ayo dukung indonesia pulih kembali : capaian g...
6668,[1448798994965041163],@tonangardyanto tapi gak harus dipaksa vaksin ...
7915,[1429953122659561487],@commuterline min naik kereta hari ini syaratn...
1105,[1349132191133954051],"dinyatakan positif, h-beberapa jam sebelum men..."
5891,[1449510102843604997],@kemenkesri mohon info lokasi vaksin dosis ke ...


In [ ]:
df.dropna(inplace=True)

In [ ]:
# save filtered data
df.to_csv(path+"filtered-tweet-github-data.csv", index=False)

In [12]:
df = df[~(df['tweet'].isna())]

In [25]:
df_chunk = pd.read_csv(path+"filtered-tweet-github-arkha-data.csv", chunksize=10000)

# Data Preprocessing

In [4]:
def clean_data(tweet):
    # lowercase
    normal_tw = tweet.lower()
    # hapus b'
    normal_tw = re.sub(r'^b\'', '', normal_tw)
    # hapus RT
    normal_tw = re.sub(r'^rt ', '', normal_tw)
    # hapus emoji
    normal_tw = re.sub(r'\\x.{2}', '', normal_tw)
    # hapus www.* atau https?://* (URL)
    normal_tw = re.sub(r'((www\.[^\s]*)|(https?://[^\s]*))', '', normal_tw)
    # remove spasi berlebih
    normal_tw = re.sub(r'\s+', ' ', normal_tw)
    # trim depan belakang
    normal_tw = normal_tw.strip()
    # regex huruf yang berulang kaya haiiii (untuk fitur unigram)
    normal_regex = re.compile(r"(.)\1{1,}")
    # buang huruf yang berulang
    normal_tw = normal_regex.sub(r"\1\1", normal_tw)
    # hapus @username
    normal_tw = re.sub(r'@[^\s]+', '', normal_tw)
    # hapus hashtag
    normal_tw = re.sub(r'#[^\s]+', '', normal_tw)
    # hapus tanda baca
    normal_tw = re.sub(r'[^\w\s]', '', normal_tw) 
    # hapus angka
    normal_tw = re.sub(r'\d+', ' ', normal_tw) 
    return normal_tw

def remove_stopwords_and_normalize(tweet):
    token = nltk.word_tokenize(tweet)
    token_new = []
    for k in token:
        if k in df_kamus_singkatan['singkatan'].values:
            k = df_kamus_singkatan.loc[df_kamus_singkatan['singkatan']
                                       == k, 'asli'].values[0]
        if k in df_kamus_alay['slang'].values:
            k = df_kamus_alay.loc[df_kamus_alay['slang']
                                  == k, 'formal'].values[0]
        if k not in stopwords[0].values:
            token_new.append(k)

    str_clean = ' '.join(token_new)
    return str_clean

def pra_pemrosesan(list_tweet, use_stopword=True):
    tweet_clean = []
    for tw in tqdm(list_tweet):
        normal_tweet = clean_data(tw)
        if use_stopword:
          normal_tweet = remove_stopwords_and_normalize(normal_tweet)
        tweet_clean.append(normal_tweet)
    return tweet_clean

In [6]:
# Mengambil data
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rFLB_1QpZhKyWFGxqPo5znTEkh7dUhPf' -O dataset.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xEIQwYre1SR71uRdQuez9MDmAUIysvAG' -O stopwordsID.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=19NOzXA8Voturopg_DTuiMns3s4M2IAUz' -O kamus_singkatan.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VjgivEr1pxyRCuyhVifPnaReFz0yd8Us' -O colloquial-indonesian-lexicon.csv

In [14]:
raw_tweet = df['tweet']
clean_tweet = np.array(pra_pemrosesan(raw_tweet, False))
df['clean_tweet'] = clean_tweet

100%|██████████| 144715/144715 [00:05<00:00, 24894.32it/s]


# Stance Detection

In [5]:
import pickle

In [6]:
with open(path+'tfidf-vectorizer-tweet-stance.sav', 'rb') as vect:
  vectorizer_tweet = pickle.load(vect)

with open(path+'model_semisupervised-stance.sav', 'rb') as model:
  clf = pickle.load(model)

In [26]:
list_pred = []

for df in df_chunk:
  data = df[~(df['tweet'].isna())]

  raw_tweet = data['tweet']
  clean_tweet = np.array(pra_pemrosesan(raw_tweet, False))
  data['clean_tweet'] = clean_tweet

  X_vect_unlabeled = vectorizer_tweet.transform(data['clean_tweet']).todense()

  pred = clf.predict(X_vect_unlabeled)
  list_pred += pred.tolist()

  del X_vect_unlabeled

100%|██████████| 4719/4719 [00:00<00:00, 21565.05it/s]


In [27]:
len(list_pred)

144715

In [28]:
df = pd.read_csv(path+"filtered-tweet-github-arkha-data.csv")
df = df[~(df['tweet'].isna())]
df['stance'] = list_pred

In [36]:
df.to_csv(path+"labeled-tweet-github-arkha-data.csv", index=False)

In [ ]:
X_vect_unlabeled = vectorizer_tweet.transform(df['clean_tweet']).todense()

In [ ]:
pred = clf.predict(X_vect_unlabeled)
df['stance'] = pred

In [ ]:
# save final data
df.to_csv(path+"labeled-tweet-github-data.csv", index=False)

In [ ]:
df.drop(['clean_tweet'], axis=1, inplace=True)

In [ ]:
df.to_csv(path+"filtered-tweet-github-data.csv", index=False)